In [0]:
from pyspark.sql import functions as F, Window
from pyspark.sql.functions import to_date, col, count, sum, avg, round

spark.sql("CREATE DATABASE IF NOT EXISTS gold")
spark.catalog.setCurrentDatabase("gold")

In [0]:
#Criação da tabela gold.ft_vendas_consumidor_local
pedidos = spark.table("silver.ft_pedido_total")
consumidores = spark.table("silver.ft_consumidores")

ft_vendas_consumidor_local = (
    pedidos
    .join(consumidores, pedidos.id_consumidor == consumidores.id_consumidor, "inner")
    .select(
        pedidos.id_pedido,
        pedidos.id_consumidor,
        F.col("valor_total_pago_brl").alias("valor_total_pedido_brl"),
        consumidores.cidade,
        consumidores.estado,
        pedidos.data_pedido
    )
)

ft_vendas_consumidor_local.write.mode("overwrite").saveAsTable("gold.ft_vendas_consumidor_local")

# View gold.view_total_compras_por_consumidor
spark.sql("""
CREATE OR REPLACE VIEW gold.view_total_compras_por_consumidor AS
SELECT 
    cidade,
    estado,
    COUNT(id_pedido) AS quantidade_vendas,
    SUM(valor_total_pedido_brl) AS valor_total_localidade
FROM gold.ft_vendas_consumidor_local
GROUP BY cidade, estado
ORDER BY valor_total_localidade DESC
""")
# Consulta por estado
spark.sql("""
SELECT 
    estado,
    SUM(quantidade_vendas) AS total_vendas,
    SUM(valor_total_localidade) AS valor_total
FROM gold.view_total_compras_por_consumidor
GROUP BY estado
ORDER BY valor_total DESC
""").display()

In [0]:
#Criação da tabela gold.ft_atrasos_pedidos_local_vendedor
pedidos = spark.table("silver.ft_pedidos")
consumidores = spark.table("silver.ft_consumidores")
itens = spark.table("silver.ft_itens_pedidos")

ft_atrasos = (
    pedidos
    .join(consumidores, "id_consumidor", "inner")
    .join(itens.select("id_pedido", "id_vendedor").dropDuplicates(["id_pedido"]), "id_pedido", "inner")
    .select(
        pedidos.id_pedido,
        itens.id_vendedor,
        pedidos.id_consumidor,
        pedidos.entrega_no_prazo,
        pedidos.tempo_entrega_dias,
        pedidos.tempo_entrega_estimado_dias,
        consumidores.cidade,
        consumidores.estado
    )
)

ft_atrasos.write.mode("overwrite").saveAsTable("gold.ft_atrasos_pedidos_local_vendedor")

# View gold.view_tempo_medio_entrega_localidade
spark.sql("""
CREATE OR REPLACE VIEW gold.view_tempo_medio_entrega_localidade AS
SELECT 
    cidade,
    estado,
    AVG(tempo_entrega_dias) AS tempo_medio_entrega,
    AVG(tempo_entrega_estimado_dias) AS tempo_medio_estimado,
    CASE 
        WHEN AVG(tempo_entrega_dias) > AVG(tempo_entrega_estimado_dias) THEN 'SIM'
        ELSE 'NÃO'
    END AS entrega_maior_que_estimado
FROM gold.ft_atrasos_pedidos_local_vendedor
WHERE tempo_entrega_dias IS NOT NULL
GROUP BY cidade, estado
ORDER BY tempo_medio_entrega DESC
""")
# View gold.view_vendedor_pontualidade
spark.sql("""
CREATE OR REPLACE VIEW gold.view_vendedor_pontualidade AS
SELECT 
    id_vendedor,
    COUNT(*) AS total_pedidos,
    SUM(CASE WHEN entrega_no_prazo = 'Não' THEN 1 ELSE 0 END) AS total_atrasados,
    ROUND(
        (SUM(CASE WHEN entrega_no_prazo = 'Não' THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 
        2
    ) AS percentual_atraso
FROM gold.ft_atrasos_pedidos_local_vendedor
GROUP BY id_vendedor
ORDER BY percentual_atraso DESC
""")

In [0]:
# Criação da gold.dm_tempo

min_max = spark.table("silver.ft_pedidos").agg(
    F.min("pedido_compra_timestamp").alias("min_date"),
    F.max("pedido_compra_timestamp").alias("max_date")
).first()

min_date = min_max.min_date.date()
max_date = min_max.max_date.date()

dm_tempo = (
    spark.createDataFrame([(min_date, max_date)], ["start", "end"])
    .select(F.explode(F.expr("sequence(start, end, interval 1 day)")).alias("sk_tempo"))
    .withColumn("ano", F.year("sk_tempo"))
    .withColumn("trimestre", F.quarter("sk_tempo"))
    .withColumn("mes", F.month("sk_tempo"))
    .withColumn("semana_do_ano", F.weekofyear("sk_tempo"))
    .withColumn("dia", F.dayofmonth("sk_tempo"))
    .withColumn("dia_da_semana_num", F.dayofweek("sk_tempo"))
    .withColumn("dia_da_semana_nome", 
        F.expr("""
            CASE dayofweek(sk_tempo)
                WHEN 1 THEN 'Domingo'
                WHEN 2 THEN 'Segunda-feira'
                WHEN 3 THEN 'Terça-feira'
                WHEN 4 THEN 'Quarta-feira'
                WHEN 5 THEN 'Quinta-feira'
                WHEN 6 THEN 'Sexta-feira'
                WHEN 7 THEN 'Sábado'
            END
        """)
    )
    .withColumn("mes_nome", 
        F.expr("""
            CASE month(sk_tempo)
                WHEN 1 THEN 'Janeiro'
                WHEN 2 THEN 'Fevereiro'
                WHEN 3 THEN 'Março'
                WHEN 4 THEN 'Abril'
                WHEN 5 THEN 'Maio'
                WHEN 6 THEN 'Junho'
                WHEN 7 THEN 'Julho'
                WHEN 8 THEN 'Agosto'
                WHEN 9 THEN 'Setembro'
                WHEN 10 THEN 'Outubro'
                WHEN 11 THEN 'Novembro'
                WHEN 12 THEN 'Dezembro'
            END
        """)
    )
    .withColumn("eh_fim_de_semana", 
        F.when(F.col("dia_da_semana_num").isin([1, 7]), "Sim").otherwise("Não")
    )
)

dm_tempo.write.mode("overwrite").saveAsTable("gold.dm_tempo")

# Criação da gold.ft_vendas_geral
pedidos = spark.table("silver.ft_pedidos")
itens = spark.table("silver.ft_itens_pedidos")
cotacao = spark.table("silver.dm_cotacao_dolar")
avaliacoes = spark.table("silver.ft_avaliacoes_pedidos")

avg_avaliacoes = (
    avaliacoes
    .groupBy("id_pedido")
    .agg(F.avg("avaliacao").alias("avaliacao_pedido"))
)

ft_vendas_geral = (
    itens
    .join(pedidos, "id_pedido", "inner")
    .join(cotacao, F.to_date(pedidos.pedido_compra_timestamp) == cotacao.data, "left")
    .join(avg_avaliacoes, "id_pedido", "left")
    .select(
        itens.id_pedido,
        itens.id_item,
        pedidos.id_consumidor.alias("fk_cliente"),
        itens.id_produto.alias("fk_produto"),
        itens.id_vendedor.alias("fk_vendedor"),
        F.to_date(pedidos.pedido_compra_timestamp).alias("fk_tempo"),
        pedidos.status.alias("status_pedido"),
        pedidos.tempo_entrega_dias,
        pedidos.entrega_no_prazo,
        itens.preco_BRL.alias("valor_produto_brl"),
        itens.preco_frete.alias("valor_frete_brl"),
        (itens.preco_BRL + itens.preco_frete).alias("valor_total_item_brl"),
        F.when(
            F.col("cotacao_dolar").isNotNull(),
            F.round(itens.preco_BRL / F.col("cotacao_dolar"), 2)
        ).alias("valor_produto_usd"),
        F.when(
            F.col("cotacao_dolar").isNotNull(),
            F.round(itens.preco_frete / F.col("cotacao_dolar"), 2)
        ).alias("valor_frete_usd"),
        F.when(
            F.col("cotacao_dolar").isNotNull(),
            F.round((itens.preco_BRL + itens.preco_frete) / F.col("cotacao_dolar"), 2)
        ).alias("valor_total_item_usd"),
        F.col("cotacao_dolar").cast("decimal(8,4)"),
        F.round(F.col("avaliacao_pedido"), 2).alias("avaliacao_pedido")
    )
)

ft_vendas_geral.write.mode("overwrite").saveAsTable("gold.ft_vendas_geral")

# Criação da view gold.view_vendas_por_periodo
spark.sql("""
CREATE OR REPLACE VIEW gold.view_vendas_por_periodo AS
SELECT 
    t.ano,
    t.trimestre,
    t.mes,
    t.mes_nome,
    t.dia,
    t.dia_da_semana_num,
    COUNT(DISTINCT v.id_pedido) AS total_pedidos,
    COUNT(v.id_item) AS total_itens,
    SUM(v.valor_total_item_brl) AS receita_total_brl,
    SUM(v.valor_total_item_usd) AS receita_total_usd,
    AVG(v.valor_total_item_brl) AS ticket_medio_brl,
    AVG(v.avaliacao_pedido) AS avaliacao_media
FROM gold.ft_vendas_geral v
INNER JOIN gold.dm_tempo t ON v.fk_tempo = t.sk_tempo
GROUP BY t.ano, t.trimestre, t.mes, t.mes_nome, t.dia, t.dia_da_semana_num
ORDER BY t.ano, t.mes, t.dia
""")

#Criação da gold.view_top_produto
spark.sql("""
CREATE OR REPLACE VIEW gold.view_top_produto AS
SELECT 
    v.fk_produto AS id_produto,
    p.categoria_produto,
    COUNT(v.id_item) AS quantidade_vendida,
    COUNT(DISTINCT v.id_pedido) AS total_pedidos,
    SUM(v.valor_total_item_brl) AS receita_brl,
    SUM(v.valor_total_item_usd) AS receita_usd,
    AVG(v.valor_produto_brl) AS preco_medio_brl,
    AVG(v.avaliacao_pedido) AS avaliacao_media,
    AVG(p.peso_produto_gramas) AS peso_medio_gramas
FROM gold.ft_vendas_geral v
INNER JOIN silver.ft_produtos p ON v.fk_produto = p.id_produto
GROUP BY v.fk_produto, p.categoria_produto
ORDER BY receita_brl DESC
""")

#Criação da view_vendas_produtos_esteticos (usando CTE)
spark.sql("""
CREATE OR REPLACE VIEW gold.view_vendas_produtos_esteticos AS
WITH vendas_fashion AS (
    SELECT 
        v.id_pedido,
        v.id_item,
        v.fk_tempo,
        v.valor_total_item_brl,
        v.valor_total_item_usd,
        v.avaliacao_pedido,
        p.categoria_produto
    FROM gold.ft_vendas_geral v
    INNER JOIN silver.ft_produtos p ON v.fk_produto = p.id_produto
    WHERE p.categoria_produto LIKE 'fashion%'
)
SELECT 
    t.ano,
    t.mes,
    vf.categoria_produto,
    COUNT(DISTINCT vf.id_pedido) AS total_pedidos,
    COUNT(vf.id_item) AS total_itens_vendidos,
    SUM(vf.valor_total_item_brl) AS receita_total_brl,
    SUM(vf.valor_total_item_usd) AS receita_total_usd,
    AVG(vf.valor_total_item_brl) AS ticket_medio_brl,
    AVG(vf.valor_total_item_usd) AS ticket_medio_usd,
    AVG(vf.avaliacao_pedido) AS avaliacao_media
FROM vendas_fashion vf
INNER JOIN gold.dm_tempo t ON vf.fk_tempo = t.sk_tempo
GROUP BY t.ano, t.mes, vf.categoria_produto
ORDER BY t.ano, t.mes, receita_total_brl DESC
""")

# Consulta 1
spark.sql("""
SELECT 
    dia_da_semana_num,
    SUM(receita_total_brl) AS receita_total
FROM gold.view_vendas_por_periodo
GROUP BY dia_da_semana_num
ORDER BY receita_total DESC
LIMIT 1
""").display()

# Consulta 2
spark.sql("""
SELECT 
    mes,
    mes_nome,
    AVG(ticket_medio_brl) AS ticket_medio
FROM gold.view_vendas_por_periodo
WHERE ano = (SELECT MAX(ano) FROM gold.dm_tempo)
GROUP BY mes, mes_nome
ORDER BY ticket_medio DESC
LIMIT 1
""").display()

In [0]:
#spark.sql("SHOW TABLES IN gold").display()workspace.gold.view_vendas_produtos_esteticos